In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold


In [16]:
best_model = tf.keras.models.load_model('Best_models/target_2/single_models/moisture_0_040-MaxErr/')

GRAIN_TYPE = 'generated_wheat'
URL = "../Datasets/processed/" + GRAIN_TYPE + ".csv"
df = pd.read_csv(URL)
print("From USDA: ", URL)
# Convert to numpy array
df_features = df[['Freq', 
                    'd(cm)', 
                   # 'Attn', 
                    'Phase_Corr', 
                    'Permittivity_real', 
                    'Permittivity_imaginary',
                    'Type',
                    ]]

df_targets = df[['M%']]

dataset_x = df_features.to_numpy()
dataset_y = df_targets.to_numpy()

From USDA:  ../Datasets/processed/generated_wheat.csv


In [17]:

#Normalizing the data set
scaler_input = MinMaxScaler()
scaler_output = MinMaxScaler()

# Normalize the entire dataset (input features)
dataset_x_norm = scaler_input.fit_transform(dataset_x)  # Use transform, NOT fit_transform

# Normalize the entire dataset (output targets)
dataset_y_norm = scaler_output.fit_transform(dataset_y)  # Use transform, NOT fit_transform
# Perform train-test split on RAW DATA
X_train, X_test, y_train, y_test = train_test_split(dataset_x_norm, dataset_y_norm, 
                                                    test_size=0.15
                                                    ,random_state=42
                                                    )

In [18]:
Y_pred_normalized = best_model.predict(dataset_x_norm)
Y_pred = scaler_output.inverse_transform(Y_pred_normalized)
Y_actual = scaler_output.inverse_transform(dataset_y_norm)


26/26 [==============================] - 0s 357us/step


In [19]:
from sklearn.metrics import mean_squared_error, mean_absolute_error,max_error, r2_score
from sigfig import round

mc_r2_score = r2_score(y_true=Y_actual[:, 0], y_pred=Y_pred[:, 0])
print("R^2: {:#.4g}".format(mc_r2_score))
mse_mc = mean_squared_error(Y_actual[:, 0], Y_pred[:, 0], squared=True)
print('Mean Squared Error: ', "{0:.4g}".format(mse_mc))
mae_mc = mean_absolute_error(Y_actual[:, 0], Y_pred[:, 0])
print('Mean Absolute Error: ', "{0:.4g}".format(mae_mc))

sums = []
for i in range(len(Y_actual[:,0])):
    sum = Y_actual[:,0][i] - Y_pred[:,0][i]
    #print(Y_actual[:,0][i]," - ",Y_pred[:,0][i],'=',sum)
    sums.append(abs(sum))
print("Min Absolute Error: ",min(sums))
print("Max Absolute Error: ",max(sums))

R^2: 0.9996
Mean Squared Error:  0.006073
Mean Absolute Error:  0.0547
Min Absolute Error:  0.00019752502441505726
Max Absolute Error:  0.4616463851928714


/home/zeenat/anaconda3/envs/WindowVersions/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
